In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd() / 'src'))

from phise import Context
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
from scipy.constants import h, c, k, pi

plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11
print("✓ Imports effectués")

## Configuration du Contexte d'Observation

Paramètres d'observation typiques VLTI en bande L (3-4 μm) ou N (8-13 μm).

In [ ]:
# Configuration du contexte
ctx = Context.get_VLTI()
ctx.Γ = 5 * u.nm

print("Contexte d'observation :")
print(f"  ✓ Architecture : SuperKN")
print(f"  ✓ Longueur d'onde : {ctx.interferometer.λ}")
print(f"  ✓ Bande spectrale : {ctx.interferometer.Δλ}")
print(f"  ✓ Télescopes : {len(ctx.interferometer.telescopes)}")
print(f"  ✓ Aire collectrice totale : {np.sum([t.diameter**2 for t in ctx.interferometer.telescopes])}")

## Spectre d'Émission Thermique

Calcul de l'émission du ciel en fonction de la longueur d'onde utilisant la loi de Planck.

In [ ]:
# Spectre du ciel thermique
print("Calcul du spectre d'émission du ciel...\\n")

# Paramètres
T_atm = 280  # K, température typique atmosphère
wavelengths = np.linspace(2, 20, 200) * u.um
wavelengths_m = wavelengths.to(u.m).value

# Loi de Planck
numerator = 2 * h * c**2 / wavelengths_m**5
denominator = np.exp(h * c / (wavelengths_m * k * T_atm)) - 1
radiance = numerator / denominator  # W / (m^3 sr)

# Convertir en photons
energy_per_photon = h * c / wavelengths_m
photon_radiance = radiance / energy_per_photon  # photons / (m^3 s sr)

# Visualiser
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Radiance
ax1.semilogy(wavelengths.value, radiance, 'b-', linewidth=2, label='Émission du ciel')
ax1.axvline(ctx.interferometer.λ.to(u.um).value, color='r', linestyle='--', 
             linewidth=2, label=f'λ observation ({ctx.interferometer.λ.to(u.um):.1f})')
ax1.set_xlabel('Longueur d\'onde (μm)', fontsize=12)
ax1.set_ylabel('Radiance (W/(m³ sr))', fontsize=12)
ax1.set_title('Spectre d\'Émission Thermique du Ciel', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3, which='both')
ax1.legend(fontsize=11)

# Photons
ax2.semilogy(wavelengths.value, photon_radiance, 'g-', linewidth=2)
ax2.axvline(ctx.interferometer.λ.to(u.um).value, color='r', linestyle='--', linewidth=2)
ax2.fill_between(wavelengths.value, 
                   (ctx.interferometer.λ - ctx.interferometer.Δλ/2).to(u.um).value * np.ones_like(wavelengths.value),
                   (ctx.interferometer.λ + ctx.interferometer.Δλ/2).to(u.um).value * np.ones_like(wavelengths.value),
                   alpha=0.2, color='blue', label='Bande spectrale')
ax2.set_xlabel('Longueur d\'onde (μm)', fontsize=12)
ax2.set_ylabel('Radiance de photons (photons/(m³ s sr))', fontsize=12)
ax2.set_title('Flux de Photons du Ciel', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig('sky_thermal_spectrum.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Spectre calculé")
idx_obs = np.argmin(np.abs(wavelengths_m - ctx.interferometer.λ.to(u.m).value))
print(f"  À λ = {wavelengths.value[idx_obs]:.2f} μm:")
print(f"    Radiance : {radiance[idx_obs]:.2e} W/(m³ sr)")
print(f"    Photon rate : {photon_radiance[idx_obs]:.2e} photons/(m³ s sr)")

## Impact sur le Rapport Signal-sur-Bruit

Analyse de comment le bruit du ciel affecte la détectabilité des compagnes.

In [ ]:
# Impact du bruit du ciel sur SNR
print("Analyse de l'impact du bruit du ciel...\\n")

# Paramètres
contrasts = np.array([0.001, 0.01, 0.1])  # Différents contrastes
star_mag = 5
integration_time = 60  # secondes

# Flux de l'étoile
flux_star = 1e7 * 10**(-(star_mag)/2.5)  # photons/s

# Flux du ciel par télescope
# Pour VLTI 8m : collectrice ~ 50 m² total
area_collector = 50  # m²
solid_angle = (np.pi / (4*np.log(2))) * (wavelengths_m[idx_obs] / 40)**2  # sr, ~1 arcsec²
flux_sky_per_tel = photon_radiance[idx_obs] * area_collector * solid_angle  # photons/s

# Bruit thermique
dark_current = 1e5  # electrons/s (typique caméra IR)
readout_noise = 100  # electrons/s (typique)

print(f"Flux estimés (T = {integration_time}s):")
print(f"  Étoile (mag {star_mag}): {flux_star * integration_time:.2e} photons")
print(f"  Ciel : {flux_sky_per_tel * integration_time:.2e} photons")
print(f"  Dark current : {dark_current * integration_time:.2e} electrons")

# SNR pour différents contrastes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Avant et après nulling
conditions = ['Sans Nulling', 'Avec Nulling\n(rejection 100x)']
null_factors = [1, 0.01]

snr_per_contrast = {}
for null_factor in null_factors:
    snrs = []
    for c in contrasts:
        flux_planet = flux_star * c
        noise = np.sqrt((flux_sky_per_tel * null_factor + dark_current) * integration_time)
        signal = flux_planet * integration_time
        snr = signal / noise if noise > 0 else 0
        snrs.append(snr)
    snr_per_contrast[null_factor] = snrs

x = np.arange(len(contrasts))
width = 0.35
for i, (nf, label) in enumerate(zip(null_factors, conditions)):
    ax1.bar(x + i*width, snr_per_contrast[nf], width, label=label)

ax1.set_ylabel('SNR', fontsize=12)
ax1.set_title('Rapport Signal-sur-Bruit vs Contraste', fontsize=13, fontweight='bold')
ax1.set_xticks(x + width/2)
ax1.set_xticklabels([f'{c*100:.1f}%' for c in contrasts])
ax1.set_xlabel('Contraste de la Compagne', fontsize=12)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3, axis='y')
ax1.set_yscale('log')

# Amélioration du nulling
null_rejections = np.logspace(0, 3, 50)  # 1x à 1000x
snr_c_fix = 0.01  # Contraste fixe
snrs_vs_nulling = []

for null_rej in null_rejections:
    flux_planet = flux_star * snr_c_fix
    noise = np.sqrt((flux_sky_per_tel / null_rej + dark_current) * integration_time)
    signal = flux_planet * integration_time
    snr = signal / noise if noise > 0 else 0
    snrs_vs_nulling.append(snr)

ax2.loglog(null_rejections, snrs_vs_nulling, 'o-', linewidth=2, markersize=5)
ax2.axhline(5, color='r', linestyle='--', linewidth=2, label='SNR = 5 (détection)')
ax2.set_xlabel('Facteur de Rejet du Ciel', fontsize=12)
ax2.set_ylabel('SNR', fontsize=12)
ax2.set_title(f'Amélioration SNR avec Nulling\n(Contraste = {snr_c_fix*100:.1f}%, {integration_time}s)', 
               fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3, which='both')
ax2.legend(fontsize=11)

plt.tight_layout()
plt.savefig('snr_vs_nulling.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\\n✓ Analyse complétée")

## Stratégies de Réjection du Ciel

Le kernel-nuller utilise plusieurs approches pour réduire l'impact du ciel :

In [ ]:
# Résumé des stratégies
print("\\n" + "="*70)
print("STRATÉGIES DE RÉJECTION DU CIEL THERMIQUE")
print("="*70)

strategies = {
    "1. Nulling Interférométrique": {
        "description": "Destructive interférence du ciel distant",
        "rejet": "10-100x",
        "limites": "Dépendant de la baseline, chromaticité"
    },
    "2. Filtrage Spectral Étroit": {
        "description": "Limiter Δλ pour réduire flux du ciel",
        "rejet": "2-5x",
        "limites": "Réduit aussi le signal scientifique"
    },
    "3. Imagerie d'Occultation": {
        "description": "Masquer l'étoile, observer compagne direct",
        "rejet": "1000x + (contraste étoile)",
        "limites": "Perceptualité instrumentale"
    },
    "4. Soustraction de Référence": {
        "description": "Mesurer ciel seul, soustraire",
        "rejet": "10-50x",
        "limites": "Variation temporelle du ciel"
    },
    "5. Modulation Spatial (Chopping)": {
        "description": "Basculer rapidement on/off source",
        "rejet": "100-1000x",
        "limites": "Vibrations induites"
    }
}

for strategy, details in strategies.items():
    print(f"\\n{strategy}")
    for key, value in details.items():
        print(f"  {key}: {value}")

print("\\n" + "="*70)
print("\\nConclusion: Le kernel-nuller combine nulling interférométrique et filtrage")
print("spectral pour atteindre les performances requises pour l'exoplanètologie.")
print("="*70)